In [1]:
# INFO: run this cell if the below three packages are not installed
!pip install wandb
!pip install onnxruntime
!pip install -q gradio 

# Load packages

In [1]:
import pandas as pd
import numpy as np
import onnxruntime as rt
from gradio import gradio as gr

In [45]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [46]:
cat_features = ['Gender','Customer_Type','Type_of_Travel','Class']
num_features = ['Age','Flight_Distance','Seat_comfort','Departure_Arrival_time_convenient','Food_and_drink','Gate_location',
                'Inflight_wifi_service','Inflight_entertainment','Online_support','Ease_of_Online_booking','Onboard_service',
                'Leg_room_service','Baggage_handling','Checkin_service','Cleanliness','Online_boarding','Departure_Delay_in_Minutes',
                'Arrival_Delay_in_Minutes']

# Define action for interface

In [53]:
def predict_satisfaction(Gender,Customer_Type,Type_of_Travel,Class,Age,Flight_Distance,Seat_comfort,
                        Departure_Arrival_time_convenient,Food_and_drink,Gate_loaction,Inflight_wifi_service,
                        Inflight_entertainment,Online_support,Ease_of_Online_booking,On_board_service,Leg_room_service,
                        Baggage_handling,Checkin_service,Cleanliness,Online_boarding,Departure_Delay_in_Minutes,
                        Arrival_Delay_in_Minutes):

    inputs_dict = {'Gender' : Gender, 
                'Customer_Type': Customer_Type, 
                'Type_of_Travel': Type_of_Travel, 
                'Class': Class, 
                'Age': float(Age), 
                'Flight_Distance': float(Flight_Distance), 
                'Seat_comfort': float(Seat_comfort), 
                'Departure_Arrival_time_convenient': float(Departure_Arrival_time_convenient), 
                'Food_and_drink': float(Food_and_drink),
                'Gate_location': float(Gate_loaction),
                'Inflight_wifi_service' : float(Inflight_wifi_service),                
                'Inflight_entertainment': float(Inflight_entertainment), 
                'Online_support': float(Online_support), 
                'Ease_of_Online_booking': float(Ease_of_Online_booking), 
                'Onboard_service': float(On_board_service),
                'Leg_room_service': float(Leg_room_service),
                'Baggage_handling': float(Baggage_handling),
                'Checkin_service': float(Checkin_service),
                'Cleanliness': float(Cleanliness),
                'Online_boarding': float(Online_boarding),
                'Departure_Delay_in_Minutes': float(Departure_Delay_in_Minutes),
                'Arrival_Delay_in_Minutes': float(Arrival_Delay_in_Minutes)
                }

    df = pd.DataFrame(inputs_dict, index = [0])
    ## DEBUG: uncomment to see what the data looks like; 
    # print(df)

    inputs = {c: df[c].values for c in df.columns}
    for c in num_features:
        inputs[c] = inputs[c].astype(np.float32)

    for k in inputs:
        inputs[k] = inputs[k].reshape((inputs[k].shape[0], 1))
    
    # INFO: load the model into the session
    sess = rt.InferenceSession('../models/satisfaction_XGB_1.onnx')
    
    ## INFO: uncomment to understand the ONNX input and output structure
    # sess_input = sess.get_inputs()
    # sess_output = sess.get_outputs()
    # print(f"No. of inputs : {len(sess_input)}, No. of outputs : {len(sess_output)}")

    # for idx, input_ in enumerate(range(len(sess_input))):
    #     input_name = sess_input[input_].name
    #     input_shape = sess_input[input_].shape
    #     input_type = sess_input[input_].type
    #     print(f"{idx} Input name : { input_name }, Input shape : {input_shape}, \
    #     Input type  : {input_type}")  

    # INFO: run the inputs through the model and make prediction
    pred_onx = sess.run(None, inputs)
    prediction = pred_onx[0][0]
    return f'The customer is expected to be {prediction.upper()}.'

# Define input types for the interface and run the Interface

In [54]:
Gender = gr.inputs.Radio(['Male','Female'], default="Male", label="Gender")
Customer_Type = gr.inputs.Radio(['Loyal Customer','disloyal Customer'], default="Loyal Customer", label="Type of Customer")
Type_of_Travel = gr.inputs.Radio(['Business travel','Personal Travel'], default="Personal Travel", label="Type of Travel")
Class = gr.inputs.Radio(['Business','Eco', 'Eco Plus'], default="Eco", label="Class of travel")
Age = gr.inputs.Slider(minimum=1, maximum=100, default=20, label="Age")
Flight_Distance = gr.inputs.Slider(minimum=10, maximum=10000, label="Flight Distance")
Seat_comfort = gr.inputs.Dropdown(list(range(0, 6)), default=3, label="Seat comfort")
Departure_Arrival_time_convenient = gr.inputs.Dropdown(list(range(0, 6)), default=3, label="Depature/Arrivel time convenient")
Food_and_drink = gr.inputs.Dropdown(list(range(0, 6)), default=3, label="Food and Drink")
Gate_loaction = gr.inputs.Dropdown(list(range(0, 6)), default=3, label="Gate location")
Inflight_wifi_service = gr.inputs.Dropdown(list(range(0, 6)), default=3, label="Inflight wifi service")
Inflight_entertainment = gr.inputs.Dropdown(list(range(0, 6)), default=3, label="Inflight entertainment")
Online_support = gr.inputs.Dropdown(list(range(0, 6)), default=3, label="Online support")
Ease_of_Online_booking = gr.inputs.Dropdown(list(range(0, 6)), default=3, label="Ease of Online booking")
On_board_service = gr.inputs.Dropdown(list(range(0, 6)), default=3, label="On board service")
Leg_room_service = gr.inputs.Dropdown(list(range(0, 6)), default=3, label="Leg room service")
Baggage_handling = gr.inputs.Dropdown(list(range(0, 6)), default=3, label="Baggage handling")
Checkin_service = gr.inputs.Dropdown(list(range(0, 6)), default=3, label="Checkin Service")
Cleanliness = gr.inputs.Dropdown(list(range(0, 6)), default=3, label="Cleanliness")
Online_boarding = gr.inputs.Dropdown(list(range(0, 6)), default=3, label="Online Boarding")
Departure_Delay_in_Minutes = gr.inputs.Dropdown(list(range(0, 6)), default=3, label="Deapture delay in minutes")
Arrival_Delay_in_Minutes = gr.inputs.Dropdown(list(range(0, 6)), default=3, label="Arrival delay in minutes")


gr.Interface(predict_satisfaction, [Gender,Customer_Type,Type_of_Travel,Class,Age,Flight_Distance,Seat_comfort,
            Departure_Arrival_time_convenient,Food_and_drink,Gate_loaction,Inflight_wifi_service,Inflight_entertainment,
            Online_support,Ease_of_Online_booking,On_board_service,Leg_room_service,Baggage_handling,Checkin_service,
            Cleanliness,Online_boarding,Departure_Delay_in_Minutes,Arrival_Delay_in_Minutes], "text", live=False).launch(debug=True);
gr.Interface()            

Running on local URL:  http://127.0.0.1:7880/

To create a public link, set `share=True` in `launch()`.


KeyboardInterrupt: 

# END OF NOTEBOOK